In [1]:
from fishsense_api_sdk.client import Client
from fishsense_api_sdk.models.image import Image
from pathlib import Path
import asyncio
from typing import List
import numpy as np
import rawpy
import matplotlib.pyplot as plt
from skimage.util import img_as_float, img_as_ubyte
import cv2
import math
from tqdm.notebook import tqdm
from skimage.exposure import adjust_gamma, equalize_adapthist
from synology_api.filestation import FileStation
from fishsense_data_processing_workflow_worker.config import settings
import os

In [ ]:
# os.getcwd()
# os.chdir('..')

# os.getcwd()

'/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker'

In [3]:
URL = "http://localhost:8000"
DIVE_ID = 471

NAS_HOST = "e4e-nas.ucsd.edu"
NAS_PORT = 6021

In [ ]:
DATA_FOLDER = (Path("../data") / "REEF" / "data").absolute()
OUTPUT_FOLDER = (Path("../output") / "preprocess_groups_jpeg").absolute()

DATA_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [5]:
client = Client(URL)

In [6]:
filestation = FileStation(NAS_HOST, NAS_PORT, settings.e4e_nas.username, settings.e4e_nas.password, secure=True, cert_verify=False)

User logged in, new session started!


In [7]:
dive = await client.dives.get(dive_id=DIVE_ID)

dive

Dive(id=471, name='121024_Alligator2_FSL03', path='2025-03-13 Reef Data Dump/121024_Alligator_FSL03/121024_Alligator2_FSL03', dive_datetime=datetime.datetime(2024, 12, 10, 10, 18, 21, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=3, dive_slate_id=None)

In [8]:
camera = await client.cameras.get(dive.camera_id)

camera

Camera(id=3, serial_number='BJ6C85528', name='FSL-03')

In [9]:
camera_intrinsics = await client.cameras.get_intrinsics(dive.camera_id)

camera_intrinsics

In [10]:
clusters = await client.images.get_clusters(dive_id=DIVE_ID)

len(clusters), len(clusters[0].image_ids), clusters[0].image_ids[0]

(34, 4, 129397)

In [11]:
cluster_images: List[List[Image]] = []

for cluster in clusters:
    cluster_images.append(await asyncio.gather(*[client.images.get(image_id=i) for i in cluster.image_ids]))

sum(len(c) for c in cluster_images), len(cluster_images), len(cluster_images[0]), cluster_images[0][0]

(79,
 34,
 4,
 Image(id=129397, path='2025-03-13 Reef Data Dump/121024_Alligator_FSL03/121024_Alligator2_FSL03/PC100001.ORF', taken_datetime=datetime.datetime(2024, 12, 10, 9, 15, 50, tzinfo=TzInfo(0)), checksum='f31697fae3cdd06ab982ee3a8ede16fe', is_canonical=True, dive_id=471, camera_id=3))

In [12]:
def process_raw(image_path: Path):
    with image_path.open("rb") as f:
        with rawpy.imread(f) as raw:
            img = img_as_float(
                raw.postprocess(
                    gamma=(1, 1),
                    no_auto_bright=True,
                    use_camera_wb=True,
                    output_bps=16,
                    user_flip=0,
                )
            )

            hsv = cv2.cvtColor(img_as_ubyte(img), cv2.COLOR_BGR2HSV)
            _, _, val = cv2.split(hsv)

            mid = 20
            mean = np.mean(val)
            meanLog = math.log(mean)
            midLog = math.log(mid * 255)
            gamma = midLog / meanLog
            gamma = 1 / gamma

            img = adjust_gamma(img, gamma=gamma)

            img = equalize_adapthist(img)

            return img_as_ubyte(img[:, :, ::-1])

In [13]:
def rectify(img: np.ndarray, camera_matrix: np.ndarray, distortion_coefficients: np.ndarray) -> np.ndarray:
    return cv2.undistort(
        img,
        camera_matrix,
        distortion_coefficients,
    )

In [14]:
for cluster in tqdm(cluster_images):
    for idx, image in enumerate(tqdm(cluster)):
        image_path = DATA_FOLDER / image.path
        target_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

        source_nas_path = f"/fishsense_data/REEF/data/{image.path}"
        filestation.get_file(source_nas_path, "download", dest_path=str(image_path.parent))

        img = process_raw(image_path)
        img = rectify(img, camera_intrinsics.camera_matrix, camera_intrinsics.distortion_coefficients)

        height, width = img.shape[:2]

        # Define text properties
        text = f"{idx + 1}/{len(cluster)}"
        org = (width - 350, height - 75) # Bottom-right corner of the text
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 5
        color = (0, 0, 255) # Red color (BGR)
        thickness = 10
        lineType = cv2.LINE_AA

        # Put the text on the image
        cv2.putText(img, text, org, fontFace, fontScale, color, thickness, lineType)

        cv2.imwrite(target_path.as_posix(), img)

        target_nas_path = f"/fishsense_process_work/preprocess_groups_jpeg"
        filestation.upload_file(target_nas_path, str(target_path), overwrite=True)

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]








Upload Progress: 100%|██████████| 7.30M/7.30M [00:01<00:00, 6.32MB/s]



















Upload Progress: 100%|██████████| 7.20M/7.20M [00:05<00:00, 1.40MB/s]














Upload Progress: 100%|██████████| 7.09M/7.09M [00:01<00:00, 4.25MB/s]










Upload Progress: 100%|██████████| 7.13M/7.13M [00:01<00:00, 5.30MB/s]


  0%|          | 0/2 [00:00<?, ?it/s]













Upload Progress: 100%|██████████| 7.36M/7.36M [00:01<00:00, 4.12MB/s]





























Upload Progress: 100%|██████████| 7.36M/7.36M [00:13<00:00, 567kB/s] 


...








Upload Progress: 100%|██████████| 7.14M/7.14M [00:00<00:00, 7.92MB/s]






Upload Progress: 100%|██████████| 7.25M/7.25M [00:01<00:00, 7.51MB/s]









Upload Progress: 100%|██████████| 7.22M/7.22M [00:01<00:00, 7.58MB/s]







Upload Progress: 100%|██████████| 7.21M/7.21M [00:00<00:00, 9.22MB/s]


  0%|          | 0/2 [00:00<?, ?it/s]








Upload Progress: 100%|██████████| 7.29M/7.29M [00:00<00:00, 8.32MB/s]








Upload Progress: 100%|██████████| 7.08M/7.08M [00:01<00:00, 7.00MB/s]


  0%|          | 0/3 [00:00<?, ?it/s]





Upload Progress: 100%|██████████| 7.17M/7.17M [00:00<00:00, 9.15MB/s]






Upload Progress: 100%|██████████| 7.42M/7.42M [00:01<00:00, 7.43MB/s]







Upload Progress: 100%|██████████| 7.40M/7.40M [00:01<00:00, 5.68MB/s]
